<a href="https://colab.research.google.com/github/Ilaharshith/Reinforcement-Learning-/blob/main/RL_Lab_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ------------------------------------------------------------
# Behavioral Cloning Example: CartPole-v1
# Author: ChatGPT (GPT-5)
# ------------------------------------------------------------

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# ------------------------------------------------------------
# 1. Create the environment
# ------------------------------------------------------------
env = gym.make("CartPole-v1")

# ------------------------------------------------------------
# 2. Expert policy (rule-based)
# ------------------------------------------------------------
def expert_policy(obs):
    """
    Simple heuristic expert policy:
    Move in the direction where the pole is falling.
    """
    angle = obs[2]
    return 0 if angle < 0 else 1  # 0 = left, 1 = right


# ------------------------------------------------------------
# 3. Collect expert demonstrations
# ------------------------------------------------------------
expert_data = {"obs": [], "actions": []}

num_episodes = 20
for episode in range(num_episodes):
    obs, _ = env.reset()
    done = False
    while not done:
        action = expert_policy(obs)
        expert_data["obs"].append(obs)
        expert_data["actions"].append(action)
        obs, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

expert_obs = np.array(expert_data["obs"])
expert_actions = np.array(expert_data["actions"])

print(f"✅ Collected {len(expert_obs)} expert samples.")

# ------------------------------------------------------------
# 4. Define the Behavioral Cloning model
# ------------------------------------------------------------
class BCModel(nn.Module):
    def __init__(self, obs_size, action_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, action_size)
        )

    def forward(self, x):
        return self.net(x)


obs_size = env.observation_space.shape[0]
action_size = env.action_space.n
model = BCModel(obs_size, action_size)

# ------------------------------------------------------------
# 5. Train the model on expert data
# ------------------------------------------------------------
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

X = torch.tensor(expert_obs, dtype=torch.float32)
y = torch.tensor(expert_actions, dtype=torch.long)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    logits = model(X)
    loss = criterion(logits, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# ------------------------------------------------------------
# 6. Evaluate the cloned policy
# ------------------------------------------------------------
def evaluate(model, episodes=5, render=False):
    model.eval()
    rewards = []
    for ep in range(episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            obs_t = torch.tensor(obs, dtype=torch.float32)
            with torch.no_grad():
                action = torch.argmax(model(obs_t)).item()
            obs, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            done = terminated or truncated
            if render:
                env.render()
        rewards.append(total_reward)
        print(f"Episode {ep+1}: Reward = {total_reward}")
    env.close()
    print(f"\n🏆 Average Reward over {episodes} episodes: {np.mean(rewards):.2f}")

# ------------------------------------------------------------
# 7. Run evaluation
# ------------------------------------------------------------
evaluate(model, episodes=5, render=False)


✅ Collected 844 expert samples.
Epoch 5/20, Loss: 0.6803
Epoch 10/20, Loss: 0.6678
Epoch 15/20, Loss: 0.6539
Epoch 20/20, Loss: 0.6373
Episode 1: Reward = 71.0
Episode 2: Reward = 73.0
Episode 3: Reward = 64.0
Episode 4: Reward = 73.0
Episode 5: Reward = 65.0

🏆 Average Reward over 5 episodes: 69.20
